<a href="https://colab.research.google.com/github/abhayrajsoni657/-AI-Change-in-Colab-Changes/blob/main/Top_Change_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q google-generativeai pillow numpy opencv-python


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.18.1 requires huggingface_hub>=0.25.0, which is not installed.
torchtune 0.6.1 requires huggingface_hub[hf_transfer], which is not installed.
datasets 4.0.0 requires huggingface-hub>=0.24.0, which is not installed.
albumentations 2.0.8 requires opencv-python-headless>=4.9.0.80, which is not installed.
sentence-transformers 5.2.2 requires huggingface-hub>=0.20.0, which is not installed.
gradio 5.50.0 requires huggingface-hub<2.0,>=0.33.5, which is not installed.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, which is not installed.
accelerate 1.12.0 requires huggingface_hub>=0.21.0, which is not installed.
albucore 0.0.24 requires opencv-python-headless>=4.9.0.80, which is not installed.
timm 1.0.24 requires huggingface_hub, which is not installed.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, b

In [7]:
import google.generativeai as genai

GOOGLE_API_KEY = "XXXXXXXXXXX"
genai.configure(api_key=GOOGLE_API_KEY)

print("✅ API Connected")


✅ API Connected


/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


In [8]:
import os
import shutil

for folder in ["inputs", "outputs"]:
    if os.path.exists(folder):
        shutil.rmtree(folder)

os.makedirs("inputs", exist_ok=True)
os.makedirs("outputs", exist_ok=True)

print("🧹 Clean workspace ready")


🧹 Clean workspace ready


In [9]:
from google.colab import files

print("📌 Upload ALL MODEL IMAGES")
uploaded = files.upload()

for filename in uploaded.keys():
    os.rename(filename, os.path.join("inputs", filename))

print("✅ Upload complete")


📌 Upload ALL MODEL IMAGES


Saving 171123_Sculptx-11_02.jpg to 171123_Sculptx-11_02.jpg
Saving 171123_Sculptx-11_03.jpg to 171123_Sculptx-11_03.jpg
Saving 171123_Sculptx-11_04.jpg to 171123_Sculptx-11_04.jpg
Saving 171123_Sculptx-11_05.jpg to 171123_Sculptx-11_05.jpg
Saving 171123_Sculptx-11_06.jpg to 171123_Sculptx-11_06.jpg
Saving 171123_Sculptx-11_09.jpg to 171123_Sculptx-11_09.jpg
Saving 171123_Sculptx-11_11.jpg to 171123_Sculptx-11_11.jpg
Saving 171123_Sculptx-11_12.jpg to 171123_Sculptx-11_12.jpg
Saving 171123_Sculptx-11_21.jpg to 171123_Sculptx-11_21.jpg
✅ Upload complete


In [10]:
TOP_PROMPT = """
ULTRA STRICT GARMENT REPLACEMENT MODE
TRUE INPAINT – TOP ONLY

ABSOLUTE COMPOSITION LOCK:
- Preserve EXACT original pose.
- Preserve EXACT camera angle.
- Preserve EXACT framing.
- Preserve EXACT crop.
- Preserve EXACT subject scale.
- Do NOT zoom.
- Do NOT crop.
- Do NOT shift.
- Do NOT resize.
- Keep original resolution identical.
- Preserve original lighting direction and intensity.
- Preserve original shadows.
- Preserve original skin tone.
- Preserve jeans 100% unchanged.
- Do NOT modify background.

EDIT SCOPE:
Modify ONLY the existing top garment area.
Do NOT affect arms, neck, waist, or jeans.
Do NOT alter body proportions.
Do NOT smooth skin.
Do NOT retouch face.

GARMENT SPECIFICATION (IDENTICAL FOR ALL IMAGES):
- Slim fit white cotton ribbed crop top
- Sleeveless
- Straight bottom hem
- Hem positioned EXACTLY 5cm above waistband
- NEVER tucked into jeans
- Same neckline shape across all images
- Same strap width across all images
- Same garment geometry and proportions
- Same hemline alignment relative to waistband
- Clean arm openings
- No logo
- No graphics
- No transparency
- No wrinkles
- No fabric deformation
- Matte premium cotton fabric
- Visible fine ribbed vertical texture
- Subtle realistic cotton micro-texture
- Physically accurate fabric tension around chest and waist

REALISM MODE:
Professional fashion catalog photography.
RAW DSLR image.
85mm lens.
f/2.8 aperture.
Ultra sharp focus.
Natural skin texture.
High dynamic range.
Physically accurate lighting.
No plastic look.
No AI artifacts.
No blur.
"""


In [11]:
import cv2
import numpy as np
from PIL import Image
import os
import google.generativeai as genai

INPUT_FOLDER = "inputs"
OUTPUT_FOLDER = "outputs"

for filename in os.listdir(INPUT_FOLDER):

    if filename.lower().endswith((".jpg", ".jpeg", ".png")):

        print(f"Processing {filename}...")

        # Stateless model (no memory carryover)
        model = genai.GenerativeModel("gemini-3-pro-image-preview")

        original_path = os.path.join(INPUT_FOLDER, filename)
        original_img = Image.open(original_path).convert("RGB")

        width, height = original_img.size

        response = model.generate_content(
            contents=[{"role": "user", "parts": [original_img, TOP_PROMPT]}],
            generation_config={
                "temperature": 0.0,
                "top_p": 0.01,
                "top_k": 1
            }
        )

        edited_bytes = response.candidates[0].content.parts[0].inline_data.data

        img_array = np.frombuffer(edited_bytes, np.uint8)
        edited_img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

        # 🔒 Force original resolution
        edited_img = cv2.resize(
            edited_img,
            (width, height),
            interpolation=cv2.INTER_NEAREST
        )

        final_path = os.path.join(OUTPUT_FOLDER, filename)
        cv2.imwrite(final_path, edited_img)

print("✅ Prompt-based batch complete.")


Processing 171123_Sculptx-11_04.jpg...
Processing 171123_Sculptx-11_12.jpg...
Processing 171123_Sculptx-11_02.jpg...
Processing 171123_Sculptx-11_21.jpg...
Processing 171123_Sculptx-11_03.jpg...
Processing 171123_Sculptx-11_06.jpg...
Processing 171123_Sculptx-11_11.jpg...
Processing 171123_Sculptx-11_09.jpg...
Processing 171123_Sculptx-11_05.jpg...
✅ Prompt-based batch complete.


In [12]:
import shutil
from google.colab import files

shutil.make_archive("Final_Outputs", 'zip', "outputs")
files.download("Final_Outputs.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>